# Capstone Project

## Introduction

A family living in the center of Düsseldorf, Germany would like to move to the outskirts of the City and needs to evaluate which neighborhood will provide them similar venues. 

For this they contacted a friend, who is currently studying the Data Science specialization, with the hope she can provide them of some ideas of where to start looking. 

Once she understood the problem. She remembered she had a module on Foursquare, where they employed K-means to cluster neighborhoods for Manhathan and Toronto. She started to look at the exercises of for that course and came up with the following results.

## Data requirements

The data requirements to solve this problem will be the neighborhoods from Düsseldorf, latitude and longitude and the zip codes known as "Postleitzahl".
For this our data scientist found the required information in this web site: http://postleitzahlen.woxikon.de/plz/duesseldorf

She needed to use the package BeatifulSoup to scrap the information. Then she changed the column names and save it as dataframe:

In [20]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("http://postleitzahlen.woxikon.de/plz/duesseldorf")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]

df.columns = ['Postalcode','District','Street']
print(df.shape)
df.to_csv('df.csv')
df.head()

(3079, 3)


,Postalcode,District,Street
0,40210,Düsseldorf Stadtmitte,Oststr.
1,40210,Düsseldorf Stadtmitte,Steinstr.
2,40210,Düsseldorf Stadtmitte,Oststr.
3,40210,Düsseldorf Stadtmitte,Marienstr.
4,40210,Düsseldorf Stadtmitte,Platz der Deutschen Einheit


To get the Latitude and Longitude, the dataframe was uploaded to this service provider: https://csv2geo.com/

Once the csv file was ready with the Latitude and Longitude coordinates. The data was imported to this notebook. Duplicates where removed and 

In [16]:
df_dus=pd.read_csv('dus_lat_long.csv',sep=',')
df_dus=df_dus[['PostalCode','District','Street','Latitude','Longitude']]
df_dus=df_dus.drop_duplicates(['Street'])
df_dus=df_dus.drop_duplicates(['Latitude'])
df_dus=df_dus.drop_duplicates(['Longitude'])
print(df_dus.shape)
df_dus.head()

(2124, 5)


,PostalCode,District,Street,Latitude,Longitude
0,40210,Düsseldorf Stadtmitte,Oststr.,51.22191,6.78580
1,40210,Düsseldorf Stadtmitte,Steinstr.,51.22300,6.78467
2,40210,Düsseldorf Stadtmitte,Marienstr.,51.22371,6.78631
3,40210,Düsseldorf Stadtmitte,Platz der Deutschen Einheit,51.22329,6.78351
4,40210,Düsseldorf Stadtmitte,Stresemannplatz,51.21922,6.78800


Since they can be many Street in one District, Streets where grouped and the average Latitude and Longitude for each District where calculated.

In [ ]:
df_dus1=df_dus.groupby(['PostalCode','District'])['Street'].apply(lambda x: ','.join(x)).reset_index()
df_dus2=df_dus.groupby(['PostalCode','District'])['Latitude','Longitude'].mean().reset_index()

Then the name of Street column was changed to Neighborhood, the order of the columns where arranged.

In [23]:
df_dus2['Neighborhood']=df_dus1['Street']
df_dus2=df_dus2[['PostalCode','District','Neighborhood','Latitude','Longitude']]
df_dus2=df_dus2.drop_duplicates(['Latitude'])
df_dus2=df_dus2.drop_duplicates(['Longitude'])
print(df_dus2.shape)
df_dus2.head()

(107, 5)


,PostalCode,District,Neighborhood,Latitude,Longitude
0,40210,Düsseldorf Stadtmitte,"Oststr.,Steinstr.,Marienstr.,Platz der Deutsch...",51.221643,6.788295
1,40211,Düsseldorf Pempelfort,"Malkastenstr.,Louise-Dumont-Str.,Wielandstr.,C...",51.230667,6.791758
2,40211,Düsseldorf Stadtmitte,"Liesegangstr.,Leopoldstr.,Kölner Str.,Kurfürst...",51.226600,6.789675
3,40212,Düsseldorf Stadtmitte,"Wagnerstr.,Josephinenstr.,Königsallee,Königstr...",51.224387,6.781410
4,40213,Düsseldorf Altstadt,"Flinger Str.,Hunsrückenstr.,Altestadt,Andreass...",51.227703,6.773738
